<a href="https://colab.research.google.com/github/ValerioGalanti/scC/blob/main/yining/benchmarks/scvi_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet scvi-colab
import sys

from scvi_colab import install

install()

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    !pip install --quiet scrublet

INFO     scvi-colab: Installing scvi-tools.                                                                        
INFO     scvi-colab: Install successful. Testing import.                                                           


/usr/local/lib/python3.10/dist-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/usr/local/lib/python3.10/dist-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (


In [3]:
import sys
import warnings

import anndata
import numpy as np
import pandas as pd
import scanpy as sc
import scrublet as scr
import scvi

ModuleNotFoundError: No module named 'scanpy'

In [ ]:
warnings.simplefilter(action="ignore", category=FutureWarning)


sc.set_figure_params(figsize=(4, 4))
scvi.settings.seed = 94705

%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}
%config InlineBackend.figure_format='retina'

INFO: Global seed set to 94705
INFO:lightning.fabric.utilities.seed:Global seed set to 94705


# Data Preparation

In [ ]:
path_to_dataset = "drive/MyDrive/research/data/"
adata = sc.read_csv(path_to_dataset + "MouseV1_MouseALM_HumanMTG.csv")
adata

AnnData object with n_obs × n_vars = 34735 × 16024

In [ ]:
adata_ref = adata[20681: -1].copy() # MTG
adata_query = adata[12553: 20681].copy() # ALM

In [ ]:
# select the top 2k varying genes
sc.pp.highly_variable_genes(adata_ref, n_top_genes=2000, flavor='seurat_v3', subset=True)
adata_query = adata_query[:, adata_ref.var_names].copy()

In [ ]:
adata_ref.shape, adata_query.shape

((14053, 2000), (8128, 2000))

# Train on reference

In [ ]:
scvi.model.SCVI.setup_anndata(adata_ref)

In [ ]:
label34 = pd.read_csv(path_to_dataset + "MouseV1_MouseALM_HumanMTG_Labels34.csv").to_numpy()[:, 0]

In [ ]:
# append the deep annotations for the reference dataset
adata_ref.obs["label34"] = label34[20681: -1]

In [ ]:
# pretraining scvi model
arches_params = dict(
    use_layer_norm="both",
    use_batch_norm="none",
    encode_covariates=True,
    dropout_rate=0.2,
    n_layers=2,
)

vae_ref = scvi.model.SCVI(adata_ref, **arches_params)
vae_ref.train(max_epochs=50)

INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Epoch 50/50: 100%|██████████| 50/50 [03:59<00:00,  4.65s/it, v_num=1, train_loss_step=3.55e+3, train_loss_epoch=3.58e+3]

INFO: `Trainer.fit` stopped: `max_epochs=50` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 50/50: 100%|██████████| 50/50 [03:59<00:00,  4.79s/it, v_num=1, train_loss_step=3.55e+3, train_loss_epoch=3.58e+3]


In [ ]:
vae_ref.save("drive/MyDrive/research/scC/models/vae_ref_top_genes")

In [ ]:
# traing the scanvi model with labels
vae_ref_scan = scvi.model.SCANVI.from_scvi_model(
    vae_ref,
    unlabeled_category="Unknown",
    labels_key="label34",
)

vae_ref_scan.train(max_epochs=20)

INFO     Training for 20 epochs.                                                                                   


INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Epoch 20/20: 100%|██████████| 20/20 [03:31<00:00,  9.93s/it, v_num=1, train_loss_step=3.69e+3, train_loss_epoch=3.71e+3]

INFO: `Trainer.fit` stopped: `max_epochs=20` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 20/20: 100%|██████████| 20/20 [03:31<00:00, 10.58s/it, v_num=1, train_loss_step=3.69e+3, train_loss_epoch=3.71e+3]


In [ ]:
dir_path_scan = "drive/MyDrive/research/scC/models/vae_ref_scan_top_genes"
vae_ref_scan.save(dir_path_scan)

# Update with query

In [ ]:
scvi.model.SCANVI.prepare_query_anndata(adata_query, dir_path_scan)

INFO     File drive/MyDrive/research/scC/models/vae_ref_scan_top_genes/model.pt already downloaded                 
INFO     Found 100.0% reference vars in query data.                                                                


In [ ]:
vae_q = scvi.model.SCANVI.load_query_data(
    adata_query,
    dir_path_scan,
)

INFO     File drive/MyDrive/research/scC/models/vae_ref_scan_top_genes/model.pt already downloaded                 


/usr/local/lib/python3.10/dist-packages/scvi/data/_manager.py:212: UserWarning: Missing labels key label34. Filling in with unlabeled category Unknown.
  field_registry[_constants._STATE_REGISTRY_KEY] = field.transfer_field(


In [ ]:
vae_q.train(
    max_epochs=100,
    plan_kwargs=dict(weight_decay=0.0),
    check_val_every_n_epoch=10,
)

INFO     Training for 100 epochs.                                                                                  


INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Epoch 100/100: 100%|██████████| 100/100 [06:29<00:00,  3.86s/it, v_num=1, train_loss_step=4.59e+3, train_loss_epoch=4.64e+3]

INFO: `Trainer.fit` stopped: `max_epochs=100` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 100/100: 100%|██████████| 100/100 [06:29<00:00,  3.90s/it, v_num=1, train_loss_step=4.59e+3, train_loss_epoch=4.64e+3]


In [ ]:
adata_query.obs["predictions"] = vae_q.predict()

In [ ]:
dir_path_scanq = "drive/MyDrive/research/scC/models/vae_q_scan_top_genes"
vae_ref_scan.save(dir_path_scanq)

# Analyze Results

In [ ]:
# accuracy
np.mean(adata_query.obs["predictions"] == label34[12553: 20681])

0.33243110236220474